In [ ]:
import logging
import time
from datetime import datetime
from pathlib import Path

from shapely.geometry import Polygon, box
import numpy as np
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
from distributed import LocalCluster, Client

import hvplot.pandas

In [ ]:
base_path = Path('../')

In [ ]:
%%time
contiguous_us_bounding_box = box(-124.848974, 24.396308, -66.885444, 49.384358)

In [ ]:
%%time
# Save various size subsets of the zip code data
zips_1 = gpd.read_file(base_path / f'data/zip_codes/zips_1.geojson')
zips_10 = gpd.read_file(base_path / f'data/zip_codes/zips_10.geojson')
zips_100 = gpd.read_file(base_path / f'data/zip_codes/zips_100.geojson')
# zips_1000 = gpd.read_file(base_path / f'data/zip_codes/zips_1000.geojson')
# zips_10000 = gpd.read_file(base_path / f'data/zip_codes/zips_10000.geojson')
# zips_28395 = gpd.read_file(base_path / f'data/zip_codes/zips_28395.geojson')

In [5]:
contiguous_us_bounding_box.bounds[0]

-124.848974

In [16]:
import geoviews.tile_sources

In [19]:
from geoviews.tile_sources import tile_sources

In [20]:
tile_sources

{'CartoDark': :WMTS   [Longitude,Latitude],
 'CartoEco': :WMTS   [Longitude,Latitude],
 'CartoLight': :WMTS   [Longitude,Latitude],
 'CartoMidnight': :WMTS   [Longitude,Latitude],
 'StamenTerrain': :WMTS   [Longitude,Latitude],
 'StamenTerrainRetina': :WMTS   [Longitude,Latitude],
 'StamenWatercolor': :WMTS   [Longitude,Latitude],
 'StamenToner': :WMTS   [Longitude,Latitude],
 'StamenTonerBackground': :WMTS   [Longitude,Latitude],
 'StamenLabels': :WMTS   [Longitude,Latitude],
 'EsriImagery': :WMTS   [Longitude,Latitude],
 'EsriNatGeo': :WMTS   [Longitude,Latitude],
 'EsriUSATopo': :WMTS   [Longitude,Latitude],
 'EsriTerrain': :WMTS   [Longitude,Latitude],
 'EsriReference': :WMTS   [Longitude,Latitude],
 'EsriOceanBase': :WMTS   [Longitude,Latitude],
 'EsriOceanReference': :WMTS   [Longitude,Latitude],
 'OSM': :WMTS   [Longitude,Latitude],
 'Wikipedia': :WMTS   [Longitude,Latitude]}

In [33]:
displays = None
# for i, zips_subset in enumerate([zips_1, zips_10, zips_100]):#, zips_1000, zips_10000, zips_28395]:
# displays = zips_1.hvplot(geo=True, color='red')
# displays *= zips_10.hvplot(geo=True, color='yellow', line='red')
displays = zips_100.hvplot(geo=True, tiles='CartoLight', color='black', xlabel='Longitude', ylabel='Latitude', title='100 Random US Zip Codes')
displays

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [ ]:
contiguous_us_bounding_box = box(-124.848974, 24.396308, -66.885444, 49.384358)
contiguous_us_bbox_gdf = gpd.GeoDataFrame(geometry=[contiguous_us_bounding_box], crs='epsg:4326')
zips_all = gpd.read_file(base_path / 'data/zip_codes/all_zip').loc[:, ['GEOID10', 'geometry']].to_crs('epsg:4326')

In [ ]:
# Filter out Multipolygons and Zip Codes outside of Contiguous US
zips_subset = gpd.sjoin(zips_all, contiguous_us_bbox_gdf, op='within')
keep_indices = zips_subset.geometry.apply(lambda x: True if type(x)==Polygon else False)
zips_subset = zips_subset[keep_indices]